In [12]:
import twitter

def oauth_login():
    # XXX: Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'jfrYegHout1Ysl17jvBD5vJzU'
    CONSUMER_SECRET = 'uW7v5O1jIRgNYeD20InorRxjjvt2TNRUpWGdCLuScr00JX4qQh'
    OAUTH_TOKEN = '14551746-iq4AbAXRI5ZBN1GkZoyaLjdf3MFD0YoLX4rEvkLFb'
    OAUTH_TOKEN_SECRET = 'X1RcW3EGFvkorrrRoyjNPQD0fTMTO0Tzl08QhIMp2YUUa'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

# Sample usage
twitter_api = oauth_login()    

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print (twitter_api)

In [13]:
import json

def twitter_search(twitter_api, q, max_results=200, **kw):

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets and 
    # https://dev.twitter.com/docs/using-search for details on advanced 
    # search criteria that may be useful for keyword arguments
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    Clinton_statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://dev.twitter.com/docs/rate-limiting/1.1/limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError, e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        Clinton_statuses += search_results['statuses']
        
        if len(Clinton_statuses) > max_results: 
            break
            
    return Clinton_statuses

# Sample usage

twitter_api = oauth_login()

q = "HillaryClinton"
Clinton_results = twitter_search(twitter_api, q, max_results=10)

        
# Show one sample search result by slicing the list...
print (json.dumps(Clinton_results[0], indent=1))

{
 "contributors": null, 
 "truncated": false, 
 "text": "Says Mrs.#DickingBimbos.... https://t.co/wABU2F8io8", 
 "is_quote_status": true, 
 "in_reply_to_status_id": null, 
 "id": 776843251890388996, 
 "favorite_count": 0, 
 "entities": {
  "symbols": [], 
  "user_mentions": [], 
  "hashtags": [
   {
    "indices": [
     9, 
     23
    ], 
    "text": "DickingBimbos"
   }
  ], 
  "urls": [
   {
    "url": "https://t.co/wABU2F8io8", 
    "indices": [
     28, 
     51
    ], 
    "expanded_url": "https://twitter.com/HillaryClinton/status/776783575375028224", 
    "display_url": "twitter.com/HillaryClinton\u2026"
   }
  ]
 }, 
 "quoted_status_id": 776783575375028224, 
 "retweeted": false, 
 "coordinates": null, 
 "quoted_status": {
  "contributors": null, 
  "truncated": false, 
  "text": "Donald Trump looks at women and decides how their looks rate on a scale of one to 10.", 
  "is_quote_status": false, 
  "in_reply_to_status_id": 776783342700228608, 
  "id": 776783575375028224, 
  "f

In [9]:
def extract_tweet_entities(Clinton_statuses):
    
    # See https://dev.twitter.com/docs/tweet-entities for more details on tweet
    # entities

    if len(Clinton_statuses) == 0:
        return []
    
    location = [place['full_name']
                   for status in Clinton_statuses
                       for place in status['entities']['full_name'] ]


    return location

# Sample usage

q = '@HillaryClinton'

Clinton_statuses = twitter_search(twitter_api, q)

location = extract_tweet_entities(Clinton_statuses)
    
# Explore the first five items for each...

print json.dumps(location[0:5], indent=1) 


KeyError: 'full_name'

In [14]:
import twitter
from collections import Counter

def get_common_tweet_entities(statuses, entity_threshold=3):

    # Create a flat list of all tweet entities
    tweet_entities = [  e
                        for place in Clinton_statuses
                            for entity_type in extract_tweet_entities([place]) 
                                for e in entity_type 
                     ]

    c = Counter(tweet_entities).most_common()

    # Compute frequencies
    return [ (k,v) 
             for (k,v) in c
                 if v >= entity_threshold
           ]

# Sample usage

q = '@HillaryClinton'

twitter_api = oauth_login()
search_results = twitter_search(twitter_api, q, max_results=100)
common_entities = get_common_tweet_entities(search_results)

print "Most common tweet entities"
print common_entities

KeyError: 'full_name'